In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

In [6]:
train_data = torchvision.datasets.MNIST(root = './data',
                                        train=True,
                                        transform = transform,
                                        download = True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15424856.63it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 452589.93it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4286569.61it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2965524.40it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [8]:
test_data = torchvision.datasets.MNIST(root = './data',
                                        train=False,
                                        transform = transform,
                                        download = True)

In [9]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [10]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=64,  # Number of samples per batch
    shuffle=True 
)

In [11]:
test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=False
)

In [12]:
for images, labels in train_loader:
    print(images.shape, labels.shape)  # Should print [batch_size, 1, 28, 28] and [batch_size]
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [13]:
import torch.nn as nn
class DigitClassify(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28*28, 128)
        self.l2 = nn.Linear(128,64)
        self.l3 = nn.Linear(64,10)
        self.soft = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        x = x.view(-1,28*28)
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        x = self.l3(x)
        return x

In [14]:
model = DigitClassify()

In [15]:
import torch
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [22]:
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step() 
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/20], Loss: 0.2433
Epoch [2/20], Loss: 0.2340
Epoch [3/20], Loss: 0.2289
Epoch [4/20], Loss: 0.2223
Epoch [5/20], Loss: 0.2177
Epoch [6/20], Loss: 0.2045
Epoch [7/20], Loss: 0.2074
Epoch [8/20], Loss: 0.1961
Epoch [9/20], Loss: 0.2023
Epoch [10/20], Loss: 0.1964
Epoch [11/20], Loss: 0.1925
Epoch [12/20], Loss: 0.1939
Epoch [13/20], Loss: 0.1797
Epoch [14/20], Loss: 0.1919
Epoch [15/20], Loss: 0.1869
Epoch [16/20], Loss: 0.1814
Epoch [17/20], Loss: 0.1865
Epoch [18/20], Loss: 0.1787
Epoch [19/20], Loss: 0.1769
Epoch [20/20], Loss: 0.1742


In [24]:
correct = 0
total = 0

# No gradient calculation needed during evaluation
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Correct:", correct)
print("Total: ", total)

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Correct: 9427
Total:  10000
Accuracy of the model on the test images: 94.27%
